<a href="https://colab.research.google.com/github/abbeymars/AbigailMarsella-AER850Project2/blob/main/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import pandas as pd
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import layers
import matplotlib.pyplot as plt


#Step 1: Data Processing
#1.1 Define input image shape
input_shape = (500, 500, 3)  #500x500 pixels, 3 color channels (RGB)
batch_size = 32

#1.2 Establish data directories
from google.colab import drive
drive.mount('/content/drive') #mount google drive
data_path = '/content/drive/My Drive/AER850Proj2/Data'  # Path to unzipped data folder

#establish train, validation, and test directories
train = os.path.join(data_path, 'train')
validation = os.path.join(data_path, 'valid')
test = os.path.join(data_path, 'test')
#verify data paths are set correctly
print("Training directory:", train)
print("Validation directory:", validation)
print("Test directory:", test)

#1.3 Perform Data Augementation
# Training data
train_data = ImageDataGenerator(
    rescale=1/255, #scale pixel values from [0, 255] to [0, 1]
    shear_range=0.2, #shear intensity
    zoom_range=0.2, #zoom range
    horizontal_flip=True #randomly flip images horizontally
)
# Validation data (note: apply only rescaling for validation)
validation_data = ImageDataGenerator(
    rescale=1/255 #scale pixel values from [0, 255] to [0, 1]
)

#1.4 Create the Train and Validation Generators
#train generator
train_generator = train_data.flow_from_directory(
    train,
    target_size=(500, 500),      #resize images (500x500 pixels)
    batch_size=batch_size,       #batch size 32
    class_mode='categorical',    #class mode categorical
    shuffle=True                 #shuffle images, ensure randomness
)
#validation generator
validation_generator = validation_data.flow_from_directory(
    validation,
    target_size=(500, 500),      #resize images (500x500 pixels)
    batch_size=batch_size,       #batch size 32
    class_mode='categorical',    #class mode categorical
    shuffle=False                #no shuffle, doesn't need to be random
)
#verify generators are set up correctly
print("Class indices:", train_generator.class_indices)
print("Class indices:", validation_generator.class_indices)


#Step 2: Neural Network Architecture Design
#initialize
model = models.Sequential()

#Convolutional Layer 1
model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(1,1), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Convolutional Layer 2
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Convolutional Layer 3
model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Convolutional Layer 4
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1,1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Flatten layer
model.add(Flatten())

#Fully Connected Dense Layer 1
model.add(Dense(128, activation='relu'))

#Fully Connected Dense Layer 2
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))  #Dropout layer to reduce overfitting

#Output Layer
model.add(Dense(3, activation='softmax'))  #3 neurons

#Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#Display the model's architecture
model.summary()


#Step 3: Hyperparameter Analysis
#Training the model
history = model.fit(
    train_generator,  #Training data generator
    epochs=20,  #Number of epochs to train the model
    batch_size=batch_size,  #Batch size is 32
    validation_data=validation_generator,  #Validation data generator
)


#Step 4: Model Evaluation
#Plot training & validation accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training directory: /content/drive/My Drive/AER850Proj2/Data/train
Validation directory: /content/drive/My Drive/AER850Proj2/Data/valid
Test directory: /content/drive/My Drive/AER850Proj2/Data/test
Found 1942 images belonging to 3 classes.
Found 431 images belonging to 3 classes.
Class indices: {'crack': 0, 'missing-head': 1, 'paint-off': 2}
Class indices: {'crack': 0, 'missing-head': 1, 'paint-off': 2}


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_7 (Conv2D)                    │ (None, 498, 498, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 249, 249, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 247, 247, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 123, 123, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 121, 121, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 60, 60, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 58, 58, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 29, 29, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 215296)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │      27,558,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 27,954,883 (106.64 MB)

 Trainable params: 27,954,883 (106.64 MB)

 Non-trainable params: 0 (0.00 B)

KeyboardInterrupt: 